<a href="https://colab.research.google.com/github/Swastik200/30DaysOfDSA/blob/main/Genetic_Algorithm4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyMetaheuristic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 2.4 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression
from pyMetaheuristic.algorithm import genetic_algorithm
from pyMetaheuristic.utils import graphs

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp "/content/drive/MyDrive/CMAPSS.zip" "/content"
!unzip CMAPSS.zip

Archive:  CMAPSS.zip
  inflating: CMAPSS/readme.txt       
  inflating: CMAPSS/RUL_FD001.txt    
  inflating: CMAPSS/RUL_FD002.txt    
  inflating: CMAPSS/RUL_FD003.txt    
  inflating: CMAPSS/RUL_FD004.txt    
  inflating: CMAPSS/test_FD001.txt   
  inflating: CMAPSS/test_FD002.txt   
  inflating: CMAPSS/test_FD003.txt   
  inflating: CMAPSS/test_FD004.txt   
  inflating: CMAPSS/train_FD001.txt  
  inflating: CMAPSS/train_FD002.txt  
  inflating: CMAPSS/train_FD003.txt  
  inflating: CMAPSS/train_FD004.txt  
  inflating: CMAPSS/x.txt            


In [5]:
index_names = ['unit_number', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = ['sensor_{}'.format(i) for i in range(1, 22)]
col_names = index_names + setting_names + sensor_names

In [28]:
path = '/content/CMAPSS/'
df_train = pd.read_csv(path + 'train_FD004.txt', sep='\s+', header=None, index_col=False, names=col_names)
df_test = pd.read_csv(path + 'test_FD004.txt', sep='\s+', header=None, index_col=False, names=col_names)
y_test = pd.read_csv(path + 'RUL_FD004.txt', sep='\s+', header=None, index_col=False, names=['RUL'])

In [29]:
# Calculate 'RUL' column based on 'time_cycles' and 'unit_number' columns
max_cycles = df_train.groupby('unit_number')['time_cycles'].max()
df_train['RUL'] = df_train.apply(lambda row: max_cycles[row['unit_number']] - row['time_cycles'], axis=1)
# Calculate 'RUL' column based on 'time_cycles' and 'unit_number' columns
max_cycles = df_test.groupby('unit_number')['time_cycles'].max()
df_test['RUL'] = df_test.apply(lambda row: max_cycles[row['unit_number']] - row['time_cycles'], axis=1)


In [30]:
train = df_train.copy()
test = df_test.copy()
display(train)
display(test)

,unit_number,time_cycles,setting_1,setting_2,setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,RUL
0,1,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,...,2387.99,8074.83,9.3335,0.02,330,2212,100.00,10.62,6.3670,320.0
1,1,2,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,...,2387.73,8046.13,9.1913,0.02,361,2324,100.00,24.37,14.6552,319.0
2,1,3,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,...,2387.97,8066.62,9.4007,0.02,329,2212,100.00,10.48,6.4213,318.0
3,1,4,42.0000,0.8400,100.0,445.00,548.70,1341.24,1118.03,3.91,...,2388.02,8076.05,9.3369,0.02,328,2212,100.00,10.54,6.4176,317.0
4,1,5,25.0063,0.6207,60.0,462.54,536.10,1255.23,1033.59,7.05,...,2028.08,7865.80,10.8366,0.02,305,1915,84.93,14.03,8.6754,316.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61244,249,251,9.9998,0.2500,100.0,489.05,605.33,1516.36,1315.28,10.52,...,2388.73,8185.69,8.4541,0.03,372,2319,100.00,29.11,17.5234,4.0
61245,249,252,0.0028,0.0015,100.0,518.67,643.42,1598.92,1426.77,14.62,...,2388.46,8185.47,8.2221,0.03,396,2388,100.00,39.38,23.7151,3.0
61246,249,253,0.0029,0.0000,100.0,518.67,643.68,1607.72,1430.56,14.62,...,2388.48,8193.94,8.2525,0.03,395,2388,100.00,39.78,23.8270,2.0
61247,249,254,35.0046,0.8400,100.0,449.44,555.77,1381.29,1148.18,5.48,...,2388.83,8125.64,9.0515,0.02,337,2223,100.00,15.26,9.0774,1.0


,unit_number,time_cycles,setting_1,setting_2,setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,RUL
0,1,1,20.0072,0.7000,100.0,491.19,606.67,1481.04,1227.81,9.35,...,2387.78,8048.98,9.2229,0.02,362,2324,100.00,24.31,14.7007,229.0
1,1,2,24.9984,0.6200,60.0,462.54,536.22,1256.17,1031.48,7.05,...,2028.09,7863.46,10.8632,0.02,306,1915,84.93,14.36,8.5748,228.0
2,1,3,42.0000,0.8420,100.0,445.00,549.23,1340.13,1105.88,3.91,...,2387.95,8071.13,9.3960,0.02,328,2212,100.00,10.39,6.4365,227.0
3,1,4,42.0035,0.8402,100.0,445.00,549.19,1339.70,1107.26,3.91,...,2387.90,8078.89,9.3594,0.02,328,2212,100.00,10.56,6.2367,226.0
4,1,5,35.0079,0.8400,100.0,449.44,555.10,1353.04,1117.80,5.48,...,2387.87,8057.83,9.3030,0.02,333,2223,100.00,14.85,8.9326,225.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41209,248,277,41.9991,0.8401,100.0,445.00,550.30,1364.40,1129.17,3.91,...,2388.50,8112.61,9.4427,0.02,331,2212,100.00,10.53,6.2620,4.0
41210,248,278,20.0026,0.7005,100.0,491.19,608.00,1494.75,1260.88,9.35,...,2388.33,8086.83,9.2772,0.02,366,2324,100.00,24.33,14.6486,3.0
41211,248,279,34.9988,0.8413,100.0,449.44,555.92,1370.65,1130.97,5.48,...,2388.64,8100.84,9.3982,0.02,336,2223,100.00,14.69,8.8389,2.0
41212,248,280,20.0027,0.7000,100.0,491.19,608.19,1489.11,1256.25,9.35,...,2388.37,8085.24,9.2727,0.03,366,2324,100.00,24.44,14.6887,1.0


In [31]:
X_train = df_train.drop(columns=['RUL'])
y_train = df_train['RUL']
X_test = df_test.drop(columns=['RUL'])
y_test = y_test['RUL']


In [32]:
parameters = {
    'population_size': 250,
    'min_values': [-5] * len(X_train.columns),  # Adjust according to your feature space
    'max_values': [5] * len(X_train.columns),  # Adjust according to your feature space
    'generations': 10,
    'mutation_rate': 0.1,
    'elite': 1,
    'eta': 1,
    'mu': 1,
    'verbose': True,
    'start_init': None,
    'target_value': None
}

In [33]:
def target_function(variables_values):
    # Train the regression model
    model = LinearRegression()
    model.fit(X_train, y_train)
    # Predict using the trained model
    predictions = model.predict(X_test)
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    return rmse

In [34]:
# Assuming you have already loaded your dataset into variables df_train and df_test

# Define X and y for the training dataset
X_train = df_train.drop(columns=['RUL'])  # Features
y_train = df_train['RUL']  # Target variable

# Define X and y for the testing dataset
X_test = df_test.drop(columns=['RUL'])  # Features
y_test = df_test['RUL']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [35]:
ga_result = genetic_algorithm(target_function=target_function, **parameters)

Generation =  0  f(x) =  55.804477948579205
Generation =  1  f(x) =  55.804477948579205
Generation =  2  f(x) =  55.804477948579205
Generation =  3  f(x) =  55.804477948579205
Generation =  4  f(x) =  55.804477948579205
Generation =  5  f(x) =  55.804477948579205
Generation =  6  f(x) =  55.804477948579205
Generation =  7  f(x) =  55.804477948579205
Generation =  8  f(x) =  55.804477948579205
Generation =  9  f(x) =  55.804477948579205
Generation =  10  f(x) =  55.804477948579205


In [36]:
optimized_model = LinearRegression().fit(X_test, y_test)

In [37]:
rmse_test = np.sqrt(mean_squared_error(y_test, optimized_model.predict(X_test)))
r2_test = r2_score(y_test, optimized_model.predict(X_test))
mae_test = mean_absolute_error(y_test, optimized_model.predict(X_test))

In [38]:
print('RMSE (Test):', rmse_test)
print('R2 Score (Test):', r2_test)
print('MAE (Test):', mae_test)

RMSE (Test): 55.735938033050246
R2 Score (Test): 0.6136420363146287
MAE (Test): 42.70655554142011
